**计算物理学(A)作业3：傅里叶变换\&函数优化**

姓名：Your name!

学号：Your id number!

注意：请使用Python语言完成所有编程作业。允许并鼓励同学间合作，但严禁抄袭别人的代码，且严谨直接抄袭ChatGPT的代码；请直接在题干后附加代码块和必要的公式，讨论等；作图请包含x,y轴label，以及必要的图例。过多信息的缺失可能会导致丢分。

作业提交只需直接提交这个完成后的jupyter notebook，并命名为"姓名-学号-作业x"。

本次作业额外注意事项：**鼓励**大家使用`numpy.fft.fft`等相关函数！另外**第二题的(d)问和第三题的(b)(c)(d)问是附加题**，供感兴趣的同学自行探索；另外附加题溢出分数可以补全作业1、作业2失去的分数。

In [113]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = (4, 3)

**1 熟悉np.fft.fft**

本问题十分的简单，甚至手搓轮子就可以10分钟内解决。但是我们希望通过这次作业让大家搞清楚`numpy.fft.fft`这个重要函数的输入、输出接口。因此，请避免自己手搓轮子(比如手动求和$a_k e^{ikx}$)，直接调包解决如下问题：

考虑时间序列$f(t)=2\cos(2\pi t)+4\cos(3\pi t+\frac{\pi}{4})+6\cos(5\pi t+\frac{\pi}{6})$.

**(a)** 请选取$t\in[0, 12]$，取$n=8000$个点画出$f(t)$时间序列，使用`numpy.fft.fft`包进行傅里叶变换，研究输出的格式细节，说服你自己（和助教本人）你能够反推出来原信号三个频率分量的**振幅，频率**，并根据实部/虚部的信息得出**相位**。

**(b)** 下面假设我们的时间序列受到了噪声的污染: $g(t)=f(t)+5\epsilon(t)$. 其中$\epsilon(t)$是满足标准正态分布的白噪声。请画出并观察傅里叶变换的频谱(振幅的模方vs.频率)，利用这个频谱进行合理滤波(滤掉振幅小的频率分量)，最后画出滤掉白噪声的平滑的时间序列。

上述问题就是最简单的滤波的实例，对于时间序列分析是非常非常重要且常用的技术。

In [ ]:
#(a)
T=12
n=8000
def func(t):
    return 2*np.cos(2*np.pi*t)+4*np.cos(3*np.pi*t+np.pi/4)+6*np.cos(5*np.pi*t+np.pi/6)

list1 = np.linspace(0,T,n)
f = func(list1)
plt.figure(figsize=(12,4))
plt.plot(list1,f)
plt.xlabel('t')
plt.ylabel('f(t)')
plt.title('Time Sequence of f(t)')
plt.show()

F = np.fft.fft(f)
list2 = np.fft.fftfreq(n,T/n)
Am = np.abs(F)/(n//2)
phase = np.angle(F)

plt.figure(figsize=(12,4))
plt.xlim(-5,5)
plt.plot(list2,Am**2)
plt.xlabel('w')
plt.ylabel('Amplitude^2')
plt.title('Frequence Sequence of F(w)(Remove negative frequence)')
plt.show()

for i in range(len(Am)):
    if Am[i]>0.1:
        print('w={}  Amplitude={}  Phase={}'.format(list2[i],Am[i],phase[i]))
print('因为信号是实数信号，所以FFT得到的频谱是对称的，因而负频率是冗余的。但是如果信号不是实数，负频率是有意义的，表征波形的不同旋转方向。')
print('可以看出在误差允许的范围内，由输出结果可以反推出频率，相位，振幅。')
        
#（b)
g = f + 5*np.random.normal(0,1,n)
G = np.fft.fft(g)
Am2 = np.abs(G)/(n//2)
list3 = np.fft.fftfreq(n,T/n)

f = func(list1)
plt.figure(figsize=(12,4))
plt.plot(list1,g)
plt.xlabel('t')
plt.ylabel('g(t)')
plt.title('Time Sequence of g(t)')
plt.show()


plt.figure(figsize=(12,4))
plt.xlim(-10,10)
plt.plot(list3,Am2)
plt.xlabel('w')
plt.ylabel('Amplitude')
plt.title('Frequence Sequence of G(w)(Amplitude)')
plt.show()

plt.figure(figsize=(12,4))
plt.xlim(-10,10)
plt.plot(list3,Am2**2)
plt.xlabel('w')
plt.ylabel('Amplitude^2')
plt.title('Frequence Sequence of G(w)(Suqare of Amplitude)')
plt.show()
print('可以看出，振幅平方后噪音影响很小，可以取过滤阈值为0.5。')

for i in range(n):
    if Am2[i]**2<0.5:
        G[i] = 0
g_new = np.fft.ifft(G)

plt.figure(figsize=(12,4))
plt.plot(list1,g_new)
plt.xlabel('t')
plt.ylabel('$g_{new}$(t)')
plt.title('Time Sequence of $g_{new}(t)$(Filter out white noise)')
plt.show()
print('由图可见，白噪声已滤掉。')

**2 简单的一维卷积**

本问题让大家更加熟悉怎么卷（bushi；

众所周知对于两个离散序列 {$f_i, i = 0, \dots , N_f-1$} 和 {$w_j, j = 0, \dots , N_w-1$}，它们的离散卷积$g$定义为：
$$
g_n = (f * w)_n = \sum_{m=-\infty}^{\infty} f_m w_{n-m}
$$

简化起见，我们假设$f$ 和 $w$在采样范围以外的值都是0。

**(a)** 随机生成两个分别长为 $N_f$ 和 $N_w$ 的正态分布序列作为$f$和$w$，手动按定义补全`my_conv`函数返回卷积序列$g$。为了检验你的函数是否正确，请随机生成一个$N_f=100, N_w=50$，对比你的输出结果和`numpy.convolve`，以及`scipy.signal.convolve`是否一致。

**(b)** 利用任何一个上一问的卷积函数，对 **1(b)** 提到的信号$g(t)$和高斯函数$h(t)$卷积：
$$ h(t) = \frac{1}{\sqrt{\pi}t_H} e^{-(t/t_H)^2} $$

对于$t_H=0.1, 0.15, 0.2$,作出不同$t_H$的卷积结果和原信号$g(t)$的效果对比图，并讨论卷积对原信号的效果，以及不同$t_H$对卷积效果的影响。

**Hint：** 由于这里卷积的信号是准连续序列，卷积结果要记得乘以采样时间间距$dt$；另外你可以对$h(t)$作截断。

**(c)** 接下来我们固定取$N_f=N, N_w=N/2$，并研究一下时间复杂度。显然`my_conv`函数返回完整输出$g$序列的时间复杂度是$O(N^2)$。请利用Python的`time`或`timeit`等模块，
- 对于$N=10, 50, 100, 500, 1000, 2000, 4000$，测量`my_conv`函数和`numpy.convolve`函数运行卷积所需时间随数据长度$N$的变化，并一起作图展示，简要讨论你的结果。
- 对于更大的$N$的值(至少取到$2*10^5$)，测量`numpy.convolve`函数和`scipy.signal.convolve`函数运行卷积所需时间随数据长度$N$的变化，并一起作图展示，简要讨论你的结果。

**Hint**：自动测量代码运行时间的办法之一如下：

```
import time
t1 = time.time()
g = myConv(f, w)
t2 = time.time()
print(t2-t1)
```

**Bonus (d)** 本问仍然固定取$N_f=N, N_w=N/2$。你可能已经注意到了`scipy.signal.convolve`明显的高效表现，请自行查阅资料调查其原因(**Hint**：时间复杂度不同)，利用图表、公式等组织语言，参考以下内容表述你的调查结果：
- 解释`scipy.signal.convolve`在算法上的优越性，并指出其理论上的时间复杂度；
- 基于上述算法思想，编写函数`my_conv_accelerated`进行算法改进，并运行代码验证你的算法首先结果正确，然后在时间开销上和`numpy.convolve`和`scipy.signal.convolve`对比，验证其显著击败`numpy.convolve`。(祝贺！你已经能够写出优于`numpy`内置库的代码了)

卷积这个概念在信号处理中也有着极其重要的地位，对相关知识感兴趣的同学请自行探索，这里限于作业篇幅实在不便继续补充（折磨）（滑稽


In [ ]:
import scipy.signal
import time

#(a)
def my_conv(f, w):    
    Nf, Nw = len(f), len(w)
    conv = []
    for n in range(Nf + Nw - 1):
        temp=0
        for m in range(Nf):
            if (n - m >= 0 and n-m <= Nw-1):
                temp += f[m] * w[n-m]
        conv.append(temp)
    return conv


np.random.seed(0)
f = np.random.normal(size=50)
w = np.random.normal(size=100)

conv1 = my_conv(f,w)
conv2 = np.convolve(f,w)
conv3 = scipy.signal.convolve(f,w)
list11 = np.linspace(0,149,149)
plt.plot(list11,conv1)
plt.plot(list11,conv2)
plt.plot(list11,conv3)
plt.show()
print('my_conv与np.convolve比较结果:{}'.format(np.allclose(conv1,conv2)))
print('my_conv与scipy.signal.convolve比较结果:{}'.format(np.allclose(conv1,conv3)))
print('由此可见，在误差允许范围内，我的输出结果和参考值一致。')

#(b)
t_H=1
def func_h(t,t_H):
    return 1/(np.sqrt(np.pi)*t_H) * np.exp(-((t-6)/t_H)**2)
t_H_list=[0.1,0.15,0.2]
list1 = np.linspace(0,T,n)
plt.figure(figsize=(12,6))
plt.plot(list1,g,label='g(t)')
for t_H in t_H_list:
    h = func_h(list1,t_H)
    conv_h = np.convolve(g,h,mode='same')*T/n
    plt.plot(list1,conv_h,label='conv with $t_H$={}'.format(t_H))
plt.xlabel('t')
plt.ylabel('y')
plt.legend()
plt.title('Convolution of g(t) with Gaussian Functions')
plt.show()
print('由图可见，卷积可以使信号曲线变得平滑，可以有效滤掉噪音，并且t_H越大，这种效果越好。但是，随着t_H的增大，振幅越来越低，且信号趋于平坦，可能会出现一定程度的失真')

 #(c)   
N_list = [10, 50, 100, 500, 1000, 2000, 4000]
my_conv_time_list = []
npconv_time_list = []
for N in N_list:
    Nf = N
    Nw = N//2
    f = np.random.normal(size=Nf)
    w = np.random.normal(size=Nw)
    t1 = time.time()
    my_conv(f,w)
    t2 = time.time()
    np.convolve(f,w)
    t3 = time.time()
    my_conv_time_list.append(t2-t1)
    npconv_time_list.append(t3-t2)
plt.figure(figsize=(12,8))
plt.plot(N_list,my_conv_time_list,marker='o',label='my_conv')
plt.plot(N_list,npconv_time_list,marker='o',label='numpy.convolve')
plt.legend()
plt.show()

N_list2 = [5000, 10000, 30000, 50000, 100000, 150000, 200000, 500000]
scipy_time_list2 = []
npconv_time_list2 = []
for N in N_list2:
    Nf = N
    Nw = N//2
    f = np.random.normal(size=Nf)
    w = np.random.normal(size=Nw)
    t1 = time.time()
    scipy.signal.convolve(f,w)
    t2 = time.time()
    np.convolve(f,w)
    t3 = time.time()
    scipy_time_list2.append(t2-t1)
    npconv_time_list2.append(t3-t2)
plt.figure(figsize=(12,8))
plt.plot(N_list2,scipy_time_list2,marker='o',label='scipy.signal.convolve')
plt.plot(N_list2,npconv_time_list2,marker='o',label='numpy.convolve')
plt.legend()
plt.show()
print('由图可见，my_conv和np.convolve的图像都近似为抛物线，时间复杂度大概为O(n^2)量级，当n很大时，scipy.signal.convolve更高效，查资料发现是O(nlogn)量级。')


(d)scipy.signal.convolve更高效是因为在n很大时，它使用快速傅立叶变换等高效方法做的卷积，时间复杂度是O（nlogn），而numpy.convolve是直接算的卷积（但有所优化），复杂度是O（n^2）。因此我将在my_conv里引入FFT算法，将复杂度降为O(nlogn)量级。

In [ ]:
def my_conv_accelerated(f, w):
    N= len(f) + len(w) -1
    return np.fft.ifft(np.fft.fft(f,N)*np.fft.fft(w,N)).real[:N]
np.random.seed(0)
f = np.random.normal(size=50)
w = np.random.normal(size=100)

conv4 = my_conv_accelerated(f,w)
list11 = np.linspace(0,149,149)
plt.plot(list11,conv2)
plt.plot(list11,conv3)
plt.show()
print('检验结果：{}'.format(np.allclose(conv2,conv4)))
#以下是测试
N_list = [10, 50, 100, 500, 1000, 2000, 5000, 10000, 20000]
my_conv_accelerated_time_list = []
new_npconv_time_list = []
new_scipy_time_list= []
for N in N_list:
    Nf = N
    Nw = N//2
    f = np.random.normal(size=Nf)
    w = np.random.normal(size=Nw)
    t1 = time.time()
    my_conv_accelerated(f,w)
    t2 = time.time()
    np.convolve(f,w)
    t3 = time.time()
    scipy.signal.convolve(f,w)
    t4 = time.time()
    my_conv_accelerated_time_list.append(t2-t1)
    new_npconv_time_list.append(t3-t2)
    new_scipy_time_list.append(t4-t3)
plt.figure(figsize=(12,8))
plt.plot(N_list,my_conv_accelerated_time_list,marker='o',label='my_conv_accelerated')
plt.plot(N_list,new_npconv_time_list,marker='o',label='numpy.convolve')
plt.plot(N_list,new_scipy_time_list,marker='o',label='scipy.signal.convolve')
plt.legend()
plt.show()
#可以看出,在n较小时，my_conv_accelerated没有优势，甚至有时不如numpy。而在n较大时，my_conv_accelerated确实显著击败了np.convolve,但仍然比不上scipy.signal.convolve，因为后者有着更多的优化。

**3 梯度下降**

本问题想让大家应用一下最速梯度法和共轭梯度法来做一些优化。

**(a)** 考察函数$f(x,y)=x^2+12y^2+3xy+13x$，请从初猜$(0, 0)$开始，分别使用讲义中标准的最速梯度法和共轭梯度法进行迭代优化，迭代至下一步函数值减小$10^{-5}$以内即停止。请画出$f(x,y)$的热力图或等高线图(`matplotlib.pyplot.contour`或`matplotlib.pyplot.contourf`)，然后在图上用两种颜色画出两种方法的迭代路线，讨论这个结果。

**注意：接下来的(b)(c)(d)问都是附加题，做了加分，不做不扣分。**

**Bonus (b)** 你可能会发现共轭梯度法的效果十分拔群，但很遗憾：并不是所有的函数都是这么美丽的二次型；对于很多比较病态的函数，如果不对优化算法进行改进，共轭梯度反而会导致灾难性的结果。举个例子：**Rosenbrock's Banana Function**：https://en.wikipedia.org/wiki/Rosenbrock_function
$$
f(x,y)=(x-1)^2 + 100(y-x^2)^2
$$
由于这里不再是简单的二次型，对于搜索方向$\hat n_k$，每步搜索的'步长'$\alpha_k$是必须人为选取的参数，这就叫超参数(Hyperparameter)，在机器学习中名叫学习率：
$$
\vec{r}_{k+1}=\vec{r}_k - \alpha_k \vec n_k 
$$

请仍然从初猜$(0, 0)$开始，取不同数量级的常数学习率$\alpha$(在优化过程中不改变)，用最速梯度和共轭梯度进行迭代优化，作图讨论你观察到的现象。

**Hint**：函数值迭代到$10^{-4}$就可以算收敛(显然最小值是0)；可以讨论两种算法下收敛性，迭代步数受学习率的影响。

**Bonus (c)** 你可能对共轭梯度法失去了信心，但实际上学习率$\alpha$随着优化的进行应当也有动态的调整。事实上`Scipy`中内置的优化库就可以利用共轭梯度法在几十步内迭代就完成我们上述的任务。请调用`scipy.optimize`验证这件事情，并接受降维打击（笑。这个故事告诉我们，不要试图用业余水平挑战专业人士写的代码。类似的故事在实际科研中十分常见，因此虽然我们要求你手写了一些优化算法，我们仍然建议你优先借助`scipy.optimize`成为调包侠。

**Bonus (d)** 你能否结合资料对算法进行改良，缩短 **(b)** 问的迭代次数？这是一个开放性问题，任何的策略、思路或者算法都可以畅所欲言。Let your ideas fly!

In [ ]:
#（a）
H = np.array([[2,3],[3,24]])

def f1(x,y):
    return x**2+12*y**2+3*x*y+13*x

def grad_f(xx):
    x = xx[0]
    y = xx[1]
    return np.array([2*x+3*y+13,24*y+3*x])
def steepest_gradient_method(x,e=10**(-5)):
    trace = [(x[0],x[1]),]
    while True:
        r = grad_f(x)
        a = np.dot(r,r)/np.dot(r,np.dot(H,r))
        x1 = x - a*r
        trace.append((x1[0],x1[1]))
        if np.linalg.norm(x1 - x) < e:
            return np.array(trace)
        x = x1

def conjugate_gradient_method(x,e=10**(-5)):
    trace = [(x[0],x[1]),]
    r = -grad_f(x)
    p = -grad_f(x)
    while True:
        alpha = np.dot(r,p)/np.dot(p,np.dot(H,p))
        x = x + alpha*p
        trace.append((x[0],x[1]))
        r1 = r - np.dot(alpha,np.dot(H,p))
        if(np.linalg.norm(r1)<e):
            return np.array(trace)
        beta = np.dot(r1,r1)/np.dot(r,r)
        p = r1 + beta*p
        r = r1
    

list1 = steepest_gradient_method(np.array([0,0]))
list2 = conjugate_gradient_method(np.array([0,0]))
print('steepest_result={}\nsteepest_minimum_value={}'.format(list1[-1],f1(list1[-1][0],list1[-1][1])))
print('conjugate_result={}\nconjugate_minimum_value={}'.format(list2[-1],f1(list2[-1][0],list2[-1][1])))
print(np.allclose(list1[-1],list2[-1]))

list1x=list1[:,0]
list1y=list1[:,1]
list2x=list2[:,0]
list2y=list2[:,1]

x = np.linspace(-10, 5, 2000)
y = np.linspace(-5, 5, 2000)
xx, yy = np.meshgrid(x, y)
z = f1(xx,yy)
plt.figure(figsize=(12, 6))
plt.contourf(xx, yy, z, levels=50)
plt.colorbar()
plt.plot(list1x, list1y, 'o-', color='green', label='Steepest Gradient Method')
plt.plot(list2x, list2y, 'x-', color='red', label='Conjugate Gradient Method')
plt.title('Different Gradient Descend Method')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()
#可见，在误差允许范围内，两种方法给出同样的结果，并且共轭梯度法步数更少，更高效。

#(b)
def g(x,y):
    return (x-1)**2+100*(y-x**2)**2
def G(x):
    xx = x[0]
    yy = x[1]
    return g(x,y)
def grad_g(xx):
    x = xx[0]
    y = xx[1]
    return np.array([2*(x-1)+400*(x*x-y)*x,200*(y-x*x)])
def new_steepest_gradient_method(x,a,e=10**(-4)):
    trace = [(x[0],x[1]),]
    for n in range(200000):
        r = grad_g(x)
        x1 = x - a*r
        trace.append((x1[0],x1[1]))
        if np.linalg.norm(r) < e:
            print('Times = {}'.format(n))
            print('steepest_result={}\nsteepest_minimum_value={}'.format(trace[-1],g(trace[-1][0],trace[-1][1])))
            return np.array(trace)
        x = x1
    print('Time Limit')
    return np.array(trace)

def new_conjugate_gradient_method(x,a,e=10**(-4)):
    trace = [(x[0],x[1]),]
    r = -grad_g(x)
    p = -grad_g(x)
    for n in range(200000):
        x = x + a*p
        trace.append((x[0],x[1]))
        r1 = -grad_g(x)
        if(np.linalg.norm(r1)<e):
            print('Times = {}'.format(n))
            return np.array(trace)
        beta = np.dot(r1,r1)/np.dot(r,r)
        p = r1 + beta*p
        r = r1
    print('Time Limit')
    return np.array(trace)

for a in [0.00001,0.0001,0.001,0.01]:
    print('alpha={}:'.format(a))
    list3 = new_steepest_gradient_method(np.array([0,0]),a)
    list4 = new_conjugate_gradient_method(np.array([0,0]),a)
print('可见耗时非常长，迭代次数极多，alpha过大过小都会使步数变得极大')
#不太能作图，会使我的程序崩溃。

    #x = np.linspace(-5, 5, 500)
    #y = np.linspace(-5, 5, 500)
    #xx, yy = np.meshgrid(x, y)
    #z = g(xx,yy)
    #continue
    #plt.figure(figsize=(12, 6))
    #plt.contourf(xx, yy, z, levels=50, cmap='viridis')
    #plt.colorbar()
    #plt.plot(list3[:,0], list3[:,1], 'o-', color='green', label='Steepest Gradient Method')
    #plt.plot(list4[:,0], list4[:,1], 'x-', color='red', label='Conjugate Gradient Method')
    #plt.title('Different Gradient Descend Method with alpha={}'.format(a))
    #plt.xlabel('x')
    #plt.ylabel('y')
    #plt.legend()
    #plt.show()

#（c)
from scipy.optimize import minimize

trace = []  
x0 = np.array([0, 0])
ans = minimize(lambda x:g(x[0],x[1]), x0, method='CG', callback=lambda x:trace.append(x.copy()))
path = np.array(trace)
x = np.linspace(-5, 5, 500)
y = np.linspace(-5, 5, 500)
xx, yy = np.meshgrid(x, y)
z = g(xx,yy)
plt.figure(figsize=(12, 6))
plt.contourf(xx, yy, z, levels=50, cmap='viridis')
plt.colorbar()
plt.plot(path[:,0], path[:,1], 'o-', color='green')
plt.title('Conjugate Gradient Method')
plt.xlabel('x')
plt.ylabel('y')
plt.show()
print(ans)
print('可见，该方法只做了165次评估和21次迭代，速度明显加快')





(d)b中效率很低是因为使用了固定的学习率，并且只使用了函数的一阶导数信息。为了改进算法，可以使用动态的学习率，比如让学习率随着步数逐渐衰减，也可以做一个动态调整，如果下一步函数值增加，那么减小学习率，反之则增加学习率。查阅资料后决定使用AdaGrad方法。

In [118]:
#(d)
def gradient_accelerated_method(x,a,e=10**(-4)):
    trace = [(x[0],x[1]),]
    rr = np.array([0.0,0.0])
    for n in range(200000):
        r = grad_g(x)
        if np.linalg.norm(r) < e:
            print('Times = {}'.format(n))
            print('steepest_result={}\nsteepest_minimum_value={}'.format(trace[-1],g(trace[-1][0],trace[-1][1])))
            return np.array(trace)
        # 使用AdaGrad调整学习率，已经过调参
        rr += r**2
        adjusted_a = a / (np.sqrt(rr) + 11.1)
        x = x - adjusted_a * r
        trace.append(x)
    print('Time Limit')
    return np.array(trace)
gradient_accelerated_method(np.array([0,0]),0.11)
print('可见迭代次数有所减少，说明算法得到了优化，但步数依然很多，还需进一步优化')



Times = 7763
steepest_result=[0.99988955 0.9997787 ]
steepest_minimum_value=1.2216097243664232e-08
可见迭代次数有所减少，说明算法得到了优化，但步数依然很多，还需进一步优化
